# Networks: structure, evolution & processes
**Internet Analytics - Lab 2**

---

**Group:** *H*

**Names:**

* *BAFFOU Jérémy*
* *BASSETO Antoine*
* *PINTO Andrea*

---

#### Instructions

*This is a template for part 4 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [2]:
import numpy as np
import random

---

## 2.4 PageRank

### 2.4.1 Random Surfer Model

#### Exercise 2.12

In [3]:
def file_to_directed_graph(filename):
    """ Parse the adjacency list into a graph
        in the form of a dictionnary.
        
        :param filename: name of the file containing the adjacency list
        
        :return: graph in the form of a dictionnary with an array of
        adjacent nodes as the value asociated to each key
    """
    
    with open("../data/" + filename) as f:
        content = f.read().splitlines()
    
    graph = {}
    for line in content:
        c = list(map(int, line.split()))
        graph[int(c[0])] = c[1:]
        
    return graph

In [4]:
def page_rank_naive(graph, nb_iterations):
    """ Naive implementation of the page rank algorithm,
        not doing anything in particular to counteract dangling nodes
        or unconnected components in the graph.
        
        :param graph: graph to explore, in the form of a dictionnary 
        with an array of adjacent nodes as the value asociated to each key
        :param nb_iterations: the number of iterations the random walk is
        going to try to take to explore the graph
        
        :return: the score associated to each node, normalised, in the form of an array
    """
    
    scores = np.zeros(len(graph))
    current = random.choice(list(graph))
    scores[current] += 1
    
    for i in range(nb_iterations):
        
        # If the current node is a dangling one, we can just return the current result as nothing else can be done
        if not graph[current]:
            return scores / np.sum(scores)
    
        current = int(random.choice(graph[current]))
        scores[current] += 1
        
    return scores / np.sum(scores)

In [5]:
def print_info(filename, naive=False):
    """ Utility function to print some information on a given graph.
    
        :param filename: name of the file containing the adjacency list
        :param naive: whether to use the naive version of page_rank or not
    """
    graph = file_to_directed_graph(filename)
    nb_iterations = 50000

    print(f"The graph : {graph}\n")

    for i in range(10):
        if naive:
            scores = page_rank_naive(graph, nb_iterations)
        else:
            scores = page_rank(graph, nb_iterations)
        print(f"trial {i} : {scores}")
        print(f"\tranking : {np.argsort(scores)}")
        print("\n")

In [6]:
print_info("absorbing.graph", naive=True)

The graph : {0: [1, 4], 1: [], 2: [3], 3: [0, 1, 2], 4: [1]}

trial 0 : [0.33333333 0.33333333 0.         0.         0.33333333]
	ranking : [2 3 0 1 4]


trial 1 : [0.  0.2 0.4 0.4 0. ]
	ranking : [0 4 1 2 3]


trial 2 : [0.5 0.5 0.  0.  0. ]
	ranking : [2 3 4 0 1]


trial 3 : [0.5 0.5 0.  0.  0. ]
	ranking : [2 3 4 0 1]


trial 4 : [0.5 0.5 0.  0.  0. ]
	ranking : [2 3 4 0 1]


trial 5 : [0.33333333 0.33333333 0.         0.         0.33333333]
	ranking : [2 3 0 1 4]


trial 6 : [0.2 0.2 0.2 0.2 0.2]
	ranking : [0 1 2 3 4]


trial 7 : [0.14285714 0.14285714 0.28571429 0.28571429 0.14285714]
	ranking : [0 1 4 2 3]


trial 8 : [0. 1. 0. 0. 0.]
	ranking : [0 2 3 4 1]


trial 9 : [0.2 0.2 0.2 0.2 0.2]
	ranking : [0 1 2 3 4]




As the graph presents an absorbing node in node 1, if the random walk starts there node 1 will have a maximum score.

In general, we can see that the results are quite disparate from one trial to the other, because the presence of a dangling node comes down to having a variable number of iterations from one trial to the next, depending on how soon the random walk gets there.

In [7]:
print_info("components.graph", naive=True)

The graph : {0: [1], 1: [2, 3], 2: [0], 3: [2], 4: [5, 6], 5: [6], 6: [7], 7: [4]}

trial 0 : [0.        0.        0.        0.        0.2850543 0.1448171 0.2850743
 0.2850543]
	ranking : [0 1 2 3 5 4 7 6]


trial 1 : [0.28567429 0.28567429 0.28567429 0.14297714 0.         0.
 0.         0.        ]
	ranking : [4 5 6 7 3 0 1 2]


trial 2 : [0.28559429 0.28559429 0.28557429 0.14323714 0.         0.
 0.         0.        ]
	ranking : [4 5 6 7 3 2 0 1]


trial 3 : [0.         0.         0.         0.         0.28553429 0.14339713
 0.28553429 0.28553429]
	ranking : [0 1 2 3 5 4 6 7]


trial 4 : [0.         0.         0.         0.         0.28575428 0.14275714
 0.28575428 0.28573429]
	ranking : [0 1 2 3 5 7 4 6]


trial 5 : [0.         0.         0.         0.         0.28547429 0.14357713
 0.28547429 0.28547429]
	ranking : [0 1 2 3 5 4 6 7]


trial 6 : [0.28551429 0.28551429 0.28551429 0.14345713 0.         0.
 0.         0.        ]
	ranking : [4 5 6 7 3 0 1 2]


trial 7 : [0.         0.

As the graph presents two connected components, depending on where the random walk starts only one is explored. This can be observed in the above trials where there always is either the first or last four nodes with a score of 0.

Apart from this, because there are no dangling nodes, the scores for each component are quite similar from one trial to the next when they are the one being explored.

#### Exercise 2.13

In [8]:
def page_rank(graph, nb_iterations, damping_factor=0.15):
    """ Implementation of the page rank algorithm.
        
        :param graph: graph to explore, in the form of a dictionnary 
        with an array of adjacent nodes as the value asociated to each key
        :param nb_iterations: the number of iterations the random walk is
        going to try to take to explore the graph
        :param damping_factor: the probability with which the random walk with just
        start from a new random node
        
        :return: the score associated to each node, normalised, in the form of an array
    """
    scores = np.zeros(len(graph))
    current = random.choice(list(graph))
    scores[current] += 1
    
    for i in range(nb_iterations):
        if np.random.choice([True, False], p=[damping_factor, 1-damping_factor]):
            current = random.choice(list(graph))
        elif not graph[current]:
            current = random.choice(list(graph))
        else:
            current = int(random.choice(graph[current]))
        scores[current] += 1
        
    return scores / np.sum(scores)

In [9]:
print_info("absorbing.graph")

The graph : {0: [1, 4], 1: [], 2: [3], 3: [0, 1, 2], 4: [1]}

trial 0 : [0.14803704 0.33809324 0.14873703 0.21355573 0.15157697]
	ranking : [0 2 4 3 1]


trial 1 : [0.14819704 0.33769325 0.15063699 0.21385572 0.14961701]
	ranking : [0 4 2 3 1]


trial 2 : [0.14749705 0.34057319 0.14641707 0.21133577 0.15417692]
	ranking : [2 0 4 3 1]


trial 3 : [0.14757705 0.34099318 0.14659707 0.21455571 0.15027699]
	ranking : [2 0 4 3 1]


trial 4 : [0.14753705 0.34257315 0.14709706 0.21253575 0.15025699]
	ranking : [2 0 4 3 1]


trial 5 : [0.14973701 0.33861323 0.14719706 0.21327573 0.15117698]
	ranking : [2 0 4 3 1]


trial 6 : [0.14603708 0.33871323 0.15089698 0.21645567 0.14789704]
	ranking : [0 4 2 3 1]


trial 7 : [0.14887702 0.33949321 0.14785704 0.21213576 0.15163697]
	ranking : [2 0 4 3 1]


trial 8 : [0.14765705 0.33701326 0.14927701 0.21277574 0.15327693]
	ranking : [0 2 4 3 1]


trial 9 : [0.14567709 0.3398732  0.14909702 0.21547569 0.149877  ]
	ranking : [0 2 4 3 1]




The new page rank implementation allows for much more consistent scores across trials, even if some disparities can still be observed in the final ranking. The effect of the dangling node is negated.

In [10]:
print_info("components.graph")

The graph : {0: [1], 1: [2, 3], 2: [0], 3: [2], 4: [5, 6], 5: [6], 6: [7], 7: [4]}

trial 0 : [0.13833723 0.13685726 0.14079718 0.0752385  0.14245715 0.07787844
 0.1447771  0.14365713]
	ranking : [3 5 1 0 2 4 7 6]


trial 1 : [0.14285714 0.14097718 0.1447571  0.07835843 0.13453731 0.07717846
 0.14221716 0.13911722]
	ranking : [5 3 4 7 1 6 0 2]


trial 2 : [0.1398972  0.13713726 0.14335713 0.07853843 0.13885722 0.07839843
 0.14305714 0.14075718]
	ranking : [5 3 1 4 0 7 6 2]


trial 3 : [0.13853723 0.13615728 0.14097718 0.07685846 0.13971721 0.07781844
 0.14613708 0.14379712]
	ranking : [3 5 1 0 4 2 7 6]


trial 4 : [0.1401172  0.13833723 0.14387712 0.07743845 0.13769725 0.07807844
 0.14327713 0.14117718]
	ranking : [3 5 4 1 0 7 6 2]


trial 5 : [0.14473711 0.14157717 0.14769705 0.07889842 0.13385732 0.07677846
 0.13969721 0.13675726]
	ranking : [5 3 4 7 6 1 0 2]


trial 6 : [0.14149717 0.13867723 0.14447711 0.07803844 0.13819724 0.07697846
 0.14237715 0.1397572 ]
	ranking : [5 3 4 1 7 0

The new page rank implementation allows for the exploration of the totality of the graph, which negates the effect of the two separate connected components. Also, the scores are quite consistent even if as before the ranking can still vary a bit.

---

### 2.4.2 Power Iteration Method

#### Exercise 2.14: Power Iteration method

In [11]:
def file_to_google_matrix(filename, theta=0.85):
    """ Computation of the google matrix for the given graph.
        
        :param filename: name of the file containing the adjacency list
        :param theta: probability of the random walk to stay on its current
        path, 1 - theta is therefore the probability to start from a new random
        node
        
        :return: google matrix in the form of an np.array
    """
    
    with open("../data/" + filename) as f:
        content = f.read().splitlines()
    
    nb_nodes = len(content)
    g = np.zeros((nb_nodes, nb_nodes))
    
    for line in content:
        c = list(map(int, line.split()))
        outgoing_degree = len(c) - 1
        
        if outgoing_degree == 0:
            g[c[0]] = np.ones(nb_nodes) / nb_nodes
        else:
            for i in c[1:]:
                g[c[0]][i] = 1 / outgoing_degree
        
    return theta * g + (1 - theta) * np.ones((nb_nodes, nb_nodes)) / nb_nodes

In [12]:
def power_iteration(g, nb_iterations=50):
    """ Computation of the score of each node given a google matrix.
        
        :param g: google matrix
        :param nb_iterations: number of iteration used to calculate 
        the score vector
        
        :return: the score vector
    """
    
    v = np.ones(np.shape(g)[0]) / np.shape(g)[0]

    for i in range(nb_iterations):
        v = v @ g

    return v

In [13]:
def get_wikipedia_pages(id_array):
    """ Utility function to get the title of Wikipedia pages given their index
        
        :param id_array: an array containing the indexes of the pages of which we
        want the titles.
        
        :return: list of strings corresponding to the titles of the pages, given in
        the same order as in id_array
    """
    
    with open("../data/wikipedia_titles.tsv") as f:
        # Strip away the first line, because it corresponds to a legend and not to a page
        content = np.array(f.read().splitlines()[1:])
        
    # Return the selected ids and format the result to only contain page titles (and not their id)
    return list(map(lambda x: x.split(None, 1)[1], content[id_array]))

In [14]:
filename = "wikipedia.graph"
g = file_to_google_matrix(filename)
scores = power_iteration(g)
for i, t in enumerate(get_wikipedia_pages(np.argsort(scores)[-1:-11:-1])):
    print(f"rank {i + 1} : {t}")

rank 1 : United States
rank 2 : United Kingdom
rank 3 : France
rank 4 : Europe
rank 5 : Germany
rank 6 : England
rank 7 : World War II
rank 8 : Latin
rank 9 : India
rank 10 : English language


---

### 2.4.3 Gaming the system *(Bonus)*

#### Exercise 2.15 *(Bonus)*

In [15]:
def get_score_and_rank_of_page(scores, page_id):
    """ Utility function to get the score and rank of a Wikipedia page given its index
        
        :param scores: an array containing the scores of each pages
        :param page_id: the id of the page of which we want the score and rank
        
        :return: formated string with the wnated information
    """
    
    nb_of_pages = len(scores)
    
    page_title = get_wikipedia_pages([page_id])[0]
    score = scores[page_id]
    rank = np.nonzero(np.argsort(scores)[::-1] == page_id)[0][0]
    
    # Return a formated strings, with the rank going from 1 for the best to the number of pages for the worst
    return f"Page \"{page_title}\":\n\tScore: {score}\n\tRank: {rank + 1} out of {nb_of_pages}"

In [16]:
filename = "wikipedia.graph"
g = file_to_google_matrix(filename)
scores = power_iteration(g)

# ID of page "United States" as a check
page_id = 5210
print(get_score_and_rank_of_page(scores, page_id))

# ID of page "History of mathematics"
page_id = 2463
print(get_score_and_rank_of_page(scores, page_id))

Page "United States":
	Score: 0.007459087286658105
	Rank: 1 out of 5540
Page "History of mathematics":
	Score: 9.846341053223486e-05
	Rank: 2530 out of 5540


In [17]:
def cheating_google(g, scores, page_id, new_edge_budget, theta=0.85):
    """ Alteration of the given google matrix by adding new edges to boost the
        score of the given page.
        One assumption made by this algorithm is that if for one node we have, 
        for every node, a non-zero probability to go there, then it is a dangling node. This
        could be challenged by the fact that a page connected to every single page would have
        the same effect on the google matrix as a dangling node, and therefore we would misclassify
        those as dangling nodes. Because in real networks this never happens, we accept this assumption.
        
        :param g: google matrix to alter
        :param scores: scores computed with the given google matrix
        :param page_id: id of the page to boost
        :param new_edge_budget: number of new edges we are allowed to add to the 
        graph to boost the score
        :param theta: probability of the random walk to stay on its current
        path, 1 - theta is therefore the probability to start from a new random
        node, has to be the same as the one that was used to compute g in the first
        place
        
        :return: an altered google matrix
    """
    
    nb_nodes = g.shape[0]
    h_hat = g - (1 - theta) * np.ones((nb_nodes, nb_nodes)) / nb_nodes
    ranking = np.argsort(scores)[::-1]
    
    # For every best ranked pages, add an edge from there to the page we want to boost
    for i in range(new_edge_budget):
        outgoing_degree = len(np.nonzero(h_hat[ranking[i]])[0])
        
        # If the page is a dangling node (i.e. in h_hat the page is connected to all pages)
        if outgoing_degree == nb_nodes:
            # Add an edge from this dangling node to the page_id
            # i.e. the probabilty to go from this node to the selected one is 1, and to any other is 0
            line = np.zeros(nb_nodes)
            line[page_id] = 1
            h_hat[ranking[i]] = line
        else:
            # If the page is not already connected to the page we want to boost, add an edge, otherwise skip this page
            if h_hat[ranking[i]][page_id] == 0:
                h_hat[ranking[i]] *= outgoing_degree / (outgoing_degree + 1)
                h_hat[ranking[i]][page_id] += 1 / (outgoing_degree + 1)
            else:
                new_edge_budget += 1
            
    return h_hat + (1 - theta) * np.ones((nb_nodes, nb_nodes)) / nb_nodes

In [18]:
# ID of page "History of mathematics"
page_id = 2463
new_edge_budget = 300
filename = "wikipedia.graph"

g = file_to_google_matrix(filename)
scores = power_iteration(g, 50)
print("Before cheating :")
print(get_score_and_rank_of_page(scores, page_id))

print("\n")

g_cheat = cheating_google(g, scores, page_id, new_edge_budget)
scores = power_iteration(g_cheat, 50)
print(f"After cheating (by adding {new_edge_budget} new edges):")
print(get_score_and_rank_of_page(scores, page_id))

Before cheating :
Page "History of mathematics":
	Score: 9.846341053223486e-05
	Rank: 2530 out of 5540


After cheating (by adding 300 new edges):
Page "History of mathematics":
	Score: 0.005659430755510088
	Rank: 2 out of 5540
